# Audiovisual Speech Recognition
In this lab you will develop a bi-modal speech recognizer that makes use of both audio and visual features.

## Data
The data that you will use corresponds to 10 utterances of the digit "2" and 10 utterances of the digit "5". 
The feature extraction has already been done.
These features can be found in the data/ folder. 
It contains both speech features and visual lip tracking features for all utterances. 
You can read data/README.md for more details about the content of the data.

## Hidden Markov Model
The method that you will use to develop your bi-modal speech recognizer is the Hidden Markov Model(HMM):

> The HMM is based on augmenting the Markov chain. A Markov chain is a model that  tells  us  something  about  the  probabilities  of  sequences  of  random  variables,states, each of which can take on values from some set. These sets can be words, or tags, or symbols representing anything, like the weather.  A Markov chain makes a very strong assumption that if we want to predict the future in the sequence, all that matters is the current state. The states before the current state have no impact on the future except via the current state. It’s as if to predict tomorrow’s weather you could examine today’s weather but you weren’t allowed to look at yesterday’s weather.

> A Markov chain is useful when we need to compute a probability for a sequence of observable events. In many cases, however, the events we are interested in are hidden: we don’t observe them directly. For example we don’t normally observe hidden part-of-speech tags in a text. Rather, we see words, and must infer the tags from the word sequence. We call the tags hidden because they are not observed. A hidden Markov model(HMM) allows us to talk about both observed events (like words that we see in the input) and hidden events (like part-of-speech tags) that we think of as causal factors in our probabilistic model.

> An HMM is specified by the following components:
* $Q=q_1q_2...q_N$ a set of $N$ states
* $A=A_{11}...a_{ij}...a_{NN}$ a **transition probability matrix** $A$, each $a_{ij}$ representing the probability of moving from state $i$ to state $j$, s.t. $\sum^N_{j=1}a_{ij}=1$  $\forall i$
* $O=o_1o_2...o_T$ a sequence of $T$ **observations**, each one drawn from a vocabulary $V=v_1,v_2,...,v_V$
* $B=b_i(o_t)$ a sequence of **observation likelihoods**, also called **emission probabilities**, each expressing the probability of an obervation $o_t$ being generated from state $i$
* $\pi=\pi_1,\pi_2,...,\pi_N$ an **initial probability distribution** over states. $\pi_i$ being the probability that the Markov chain will start in state $i$. Some states $j$ may have $\pi_j=0$, meaning that they cannot be initial states. Also, $\sum^n_{i=1}pi_i=1$

This lab focuses on two of the fundamental HMM problems:

*  Learning: Given an observation sequence $O$ and the set of states in the HMM, learn the HMM parameters A and B. 
*  Likelihood: Given a HMM $\lambda = (A,B)$ and on observation sequence $O$, determine the likelihood $P(O|\lambda)$.

You are provided with code that implements the EM algorithm and the Forward-Backward algorithm which solve these problems.
The code is only partially complete however. It is up to you to fill in the gaps.


# Code

In [ ]:
# Imports
import numpy as np

ghmm_learn is the function that you will call to learn a HMM.

You will later use its output parameters to calculate the likelihood that a sequence belongs to that HMM.

In [ ]:
def ghmm_learn(Yseq,N,Ainit=None):
# Input:
#  N - number of states
#  Ainit  - This is the initial A matrix that you can input to force the model
#           to learn a left to right model.
#  Yseq - This is the data. 

# Output:
#  P0 - Initial state probability matrix
#  A - Transition probability matrix
#  mu - Mean matrix
#  sigma - Covariance matrix 

    M = np.shape(Yseq[0])[1]
    P0 = np.ones([N,1])/N;
    
    if Ainit is None:
        A = np.random.rand(N,N) + 2 * np.eye(N)
        for l in range(N):
            A[l,:] = A[l,:]/np.sum(A[l,:]);
    else:
        A = Ainit
        
    Nseq = len(Yseq)
    datatmp = Yseq[0]
    
    for seq in range(1,Nseq):
        datatmp = np.append(datatmp,Yseq[seq],axis=0)
        
    M = np.shape(datatmp)[1]
    T = np.shape(datatmp)[0]
    T1 = int(np.floor(T/N))
    
    mu = np.zeros([N,M])
    sigma = np.zeros([N,M,M])

    for i in range(N):
        mu[i,:] = np.mean(datatmp[i*T1:(i+1)*T1-1,:],axis=0)
        sigma[i,:,:] = np.cov(np.transpose(datatmp))
        
    for itr in range(25):
        params = ghmm_em_obs(Yseq,N,A,P0,mu,sigma)
        mu = params[0]
        A = ghmm_em_trans(Yseq,N,A,P0,mu,sigma)
    return(P0,A,mu,sigma)

ghmm_em_obs calculates the observation mean and covariances.

You don't need to do anything yourself with this function.

In [ ]:
def ghmm_em_obs(Yseq,N,A,P0,mu,sigma):
    N_seq = np.max(np.shape(Yseq))
    eps = 1e-5
    M = np.shape(Yseq[0])[1]
    museq_unnorm = np.zeros([N,M])
    museq_norm = np.zeros([N,M])
    sigmaseq_unnorm = np.zeros([N,M,M])
    sigmaseq_norm = np.zeros([N,1])
    Pxyseq = list([] for _ in range(N_seq))
    Pxxyseq = list([] for _ in range(N_seq))
    llseq = list([] for _ in range(N_seq))
    
    for seq in range(N_seq):
        Y = Yseq[seq]
        M = np.shape(Y)[1]
        T = np.shape(Y)[0]

        fwd = ghmm_fwd(Y,A,P0,mu,sigma)
        alpha = fwd[0]
        scale = fwd[1]
        beta = ghmm_bwd(Y,A,P0,mu,sigma,scale)
        probs = ghmm_prob(alpha,beta,scale,A,mu,sigma,Y)
        Pxy = probs[0]
        Pxxy = probs[1]
        Pxyseq[seq] = Pxy
        Pxxyseq[seq] = Pxxy
        llseq[seq] = np.exp(np.sum(scale))
        
        for i in range(N):
            Pxy0 = Pxy[:,i].reshape(-1,1)
            Pxy0 = np.sum(np.multiply(np.repeat(Pxy0,M,axis=1),Y),axis=0)+eps
            museq_unnorm[i,:] = np.add(museq_unnorm[i,:],Pxy0)
            museq_norm[i,:] = np.add(museq_norm[i,:],np.sum(Pxy[:,i])+eps)
            
    mu = np.divide(museq_unnorm,museq_norm)
    sigmaseq_unnorm = np.zeros([N,M,M])
    sigmaseq_norm = np.zeros([N,1])
    for seq in range(N_seq):
        Y = Yseq[seq]
        M = np.shape(Y)[1]
        T = np.shape(Y)[0]

        fwd = ghmm_fwd(Y,A,P0,mu,sigma)
        alpha = fwd[0]
        scale = fwd[1]
        beta = ghmm_bwd(Y,A,P0,mu,sigma,scale)
        probs = ghmm_prob(alpha,beta,scale,A,mu,sigma,Y)
        Pxy = probs[0]
        Pxxy = probs[1]
        Pxyseq[seq] = Pxy
        Pxxyseq[seq] = Pxxy
        llseq[seq] = np.exp(np.sum(scale));
    
        for i in range(N):
            Pxy0 = Pxy[:,i]
            sigmatmp = np.zeros([M,M])
            for t in range(T):
                sigmatmp = np.add(sigmatmp, np.multiply(np.transpose(Y[t,:] - mu[i,:]).reshape(-1,1), np.multiply(Y[t,:] - mu[i,:],Pxy0[t])))
                sigmaseq_norm[i] = sigmaseq_norm[i] + Pxy0[t]
            sigmaseq_unnorm[i,:,:] = sigmaseq_unnorm[i,:,:] + np.reshape(sigmatmp,[1,M,M],order='F')
            
    for i in range(N):
        sigma[i,:,:] = np.divide(sigmaseq_unnorm[i,:,:],sigmaseq_norm[i])
    return(mu,sigma)                                              

ghmm_prob calculates two types of probabilities:

First is the probability of being in state $S_i$ at time $t$, given the observation sequence $O$ and the model $\lambda$:

$\gamma_t(i)= P(q_t=S_i|0,\lambda) = \frac{\alpha_t(i)\beta_t(i)}{P(O|\lambda)} = \frac{\alpha_t(i)\beta_t(i)}{\sum_{i=1}^N\alpha_t(i)\beta_t(i)}$

You will need to insert a line of code into this function that computes p(x|y) from p(x,y) where x is the state index and y is the observation.

Second is the probability of being in state $S_i$ ad time $t$ and $S_j$ at time $t+1$, given the observation sequence $O$ and the model $\lambda$:

$\xi_t(i,j)= P(q_t=S_i,q_{t+1}=S_j|0,\lambda) = \frac{\alpha_t(i)a_{ij}b_j(O_{t+1})\beta_t(i)}{P(O|\lambda)} = \frac{\alpha_t(i)a_{ij}b_j(O_{t+1})\beta_t(i)}{\sum_{i=1}^N\sum_{j=1}^N\alpha_t(i)a_{ij}b_j(O_{t+1})\beta_t(i)}$

You will need to insert another line of code into this function that computes p(x(t),x(t+1)|Y) from p(x(t),x(t+1),Y).

In [ ]:
def ghmm_prob(alpha,beta,scale,A,mu,sigma,Y):
# Input:
#   Y - observation vector sequence 
#   A - state transition probabilities
#   P0 - initial state probabilities 
#   mu - observation mean
#   sigma - observation variance: diagonal

# Output:
#  Pxy = P(x_t|Y)
#  Pxxy = P(x_t,x_{t+1}|Y)

    scale = np.exp(scale)
    N = np.shape(A)[0]
    i = np.ones([1,N])
    T = np.shape(Y)[0]
    M = np.shape(Y)[1]
    
    Pys = np.zeros([N,T])

    for j in range(N):
        sigmai = np.reshape(sigma[j,:,:],(M,M),order='F')
        for t in range(T):
            Pys[j,t] = np.sqrt(1/(((2*np.pi)**M)*np.linalg.det(sigmai)))*np.exp(np.dot(-0.5*(Y[t,:]-mu[j,:]),np.dot(np.linalg.inv(sigmai),(Y[t,:]-mu[j,:]).reshape(-1,1))))
    Pxy = np.zeros([T,N])
    for t in range(T):
        Pxy0 = np.multiply(alpha[:,t],np.exp(beta[:,t])*scale[t])
        Pxy[t,:] = ???
        
    Pxxy = np.zeros([T,N,N])
    for t in range(T-1):
        Pxxy0 = np.multiply(alpha[:,t].reshape(-1,1) * np.transpose(np.multiply(np.exp(beta[:,t+1]),Pys[:,t+1])),A)
        Pxxy[t,:,:]= ???
    return(Pxy,Pxxy)

ghmm_em_trans learns the parameters of the transition matrix, using the observation mean and covariance that have been calculated by ghmm_em_obs.

You will need to insert a line of code into this function that computes the new transition probabilities A[k,m] from the corresponding soft-count matrix Na[k,m]:

$\hat{a}_{ij} = \frac{\textrm{Expected number of transitions from state S_i to state S_j}}{\textrm{expected number of transitions from state S_i}} = \frac{\sum_{t=1}^{T-1} \xi_t(i,j)}{\sum_{t=1}^{T-1}\sum_{k=1}^N \xi_t(i,k)}$

In [ ]:
def ghmm_em_trans(Yseq,N,A,P0,mu,sigma):
    N_seq = np.max(np.shape(Yseq))
    eps = 0
    MAX_ITR = 20
    Naseq = np.zeros([N,N])
    
    for seq in range(N_seq):
        Y = Yseq[seq]
        M = np.shape(Y)[1]
        T = np.shape(Y)[0]
        fwd = ghmm_fwd(Y,A,P0,mu,sigma)
        alpha = fwd[0]
        scale = fwd[1]
        beta = ghmm_bwd(Y,A,P0,mu,sigma,scale)
        probs = ghmm_prob(alpha,beta,scale,A,mu,sigma,Y)
        Pxy = probs[0]
        Pxxy = probs[1]
        
        Na = np.zeros([N,N])
        for i in range(N):
            for j in range(N):
                Na[i,j] = np.sum(Pxxy[:,i,j])
        
        Na = Na+eps
        Naseq = Naseq + Na
    Anew = np.zeros([N,N])
    for k in range(N):
        Anew[k,:] = ???
    return Anew

The forward step of the forward-backward algorithm. This calculates:

$\alpha_t(i)=P(O_1O_2...O_t,q_t=S_i|\lambda)$

which is the probability of a partial observation sequence $O_1O_2...O_t$ and state $S_i$ at time $t$, given the model $\lambda$.

You will need to insert a line of code into ghmm_fwd that computes alpha0 from transition probabilities A, Gaussian probabilities Pys and alpha[t-1]:

$\alpha_{t+1} = [\sum^N_{i=1} \alpha_t(i)a_{ij}] b_j (O_{t+1}), 1\leq T-1, 1\leq j \leq N$

The initialization $\alpha_1(i) = \pi_1b_i(O_1),1\leq i\leq N$ has already been done.

Once you have learned a HMM you can use this function to determine the likelihood of an observation sequence belonging to that HMM.

In [ ]:
def ghmm_fwd(Y,A,P0,mu,sigma):
# Input:
#   T - observation vector sequence
#   A = state transition probabilities
#   P0 - initial state probabilities
#   mu - observation mean
#   sigma - observation variance

# Output:
#   alpha - E[ s_t | Y_t ] = Pr( s_t | observation sequence up to time 't')
#   scale - log Pr( s_0, ..., s_t | observation sequence up to time 't')
    N = np.shape(A)[0]
    i = np.ones([1,N])
    T = np.shape(Y)[0]
    M = np.shape(Y)[1]
    alpha = np.zeros([np.shape(P0)[0],T])
    Pys = np.zeros([N,T])
    
    for j in range(N):
        sigmai = np.reshape(sigma[j,:,:],(M,M),order='F')
        for t in range(T):
            Pys[j,t] = np.sqrt(1/(((2*np.pi)**M)*np.linalg.det(sigmai)))*np.exp(np.dot(-0.5*(Y[t,:]-mu[j,:]),np.dot(np.linalg.inv(sigmai),(Y[t,:]-mu[j,:]).reshape(-1,1))))

    alpha0 = np.multiply(P0,Pys[:,0].reshape(-1,1))
    scale = np.zeros([T,1])
    scale[0] = np.sum(alpha0)    
    alpha[:,0] = (alpha0 / scale[0]).flatten()
    
    for t in range(1,T):
        # Hint: Use np.reshape(-1,1) to make a 2-d vector from a 1-d vector. 
        #   This changes how numpy handles certain operations such as '*'.
        alpha0 = ???
        scale[t] = np.sum(alpha0)
        alpha[:,t] = alpha0 / scale[t]
        
    scale = np.log(scale)
    return (alpha,scale)

The backward step of the forward-backward algorithm. This calculates:

$\beta_t(i) = P(O_{t+1}O_{t+2}...O_T|q_t=S_i,\lambda)$

which is the probability of the partial observation sequence from $t+1$ to $T$, given state $S_i$ at time $t$ and the model $\lambda$

You will need to insert a line of code into ghmm_bwd that computes beta from transition probabilities A, Gaussian probabilities Pys, and beta(t+1):

$\beta_t(i) = \sum^N_{j=1}a_{ij}b_j(O_{t+1})\beta_{t+1}(j),t=T-1,T-2,...,1, 1 \leq i \leq N$

The initialization $\beta_T(i) = 1,1\leq i\leq N$ has already been done.

In [ ]:
def ghmm_bwd(Y,A,P0,mu,sigma,scale):
# Input:
#   T - observation vector sequence
#   A = state transition probabilities
#   P0 - initial state probabilities
#   mu - observation mean
#   sigma - observation variance
#   scale - see 'ghmm_fwd'

# Output:
#   beta - Pr( s_t, observation sequence from time 't+1')

    scale = np.exp(scale)
    N = np.shape(A)[0];
    T = np.shape(Y)[0]
    M = np.shape(Y)[1]
    Pys = np.zeros([N,T])
    
    for j in range(N):
        sigmai = np.reshape(sigma[j,:,:],[M,M],order='F')
        for t in range(T):
            Pys[j,t] = np.sqrt(1/(((2*np.pi)**M)*np.linalg.det(sigmai)))*np.exp(np.dot(-0.5*(Y[t,:]-mu[j,:]),np.dot(np.linalg.inv(sigmai),(Y[t,:]-mu[j,:]).reshape(-1,1))))
            
    beta = np.zeros([N,T])
    beta[:,T-1] = (np.ones([N,1])/scale[T-1]).flatten()
    for t in range(T-2,-1,-1):
        # Hint: Use np.reshape(-1,1) to make a 2-d vector from a 1-d vector. 
        #   This changes how numpy handles certain operations such as '*'.
        beta[:,t] = ???
    beta = np.log(beta+1.0e-10)
    
    return beta

# Evaluation
Now that you can learn a HMM, you can use the learned parameters with ghmm_fwd to predict the likelihood of a sequence belonging to that HMM. 

You now have to evaluate how well the HMMs can distinguish between the utterances of the digit "2" and of the digit "5" by using the "leave one out" scheme. 

There are a total of 20 utterances. Learn a HMM for the digit "2" and another for the digit "5" using 19 utterances of the data and test it on the single utterance that you have left out. Repeat this 20 times until you have tested all 20 utterances. 

Do this procedure for:
* Only the audio features
* Only the video features
* The audio and video features combined (concatenated).

Determine the performance of each of these sets of features by calculating the accuracy: $\frac{\#correct}{20}$

In [ ]:
path = "AV_DATA"
N = 5
Ainit = np.array([[0.8,0.2,0,0,0],[0,0.8,0.2,0,0],[0,0,0.8,0.2,0],[0,0,0,0.8,0.2],[0,0,0,0,1]])

# Enter your code here